In [8]:
# ===================================================================================
# 1. CONFIGURAÇÃO DO AMBIENTE E IMPORTAÇÕES
# ===================================================================================
!pip install splink -q

import duckdb
import pandas as pd
from splink import Linker, SettingsCreator, DuckDBAPI
# Importando as bibliotecas de comparação e níveis de comparação
import splink.comparison_library as cl
import splink.comparison_level_library as cll
from splink import block_on
from splink.exploratory import completeness_chart, profile_columns
from splink.blocking_analysis import cumulative_comparisons_to_be_scored_from_blocking_rules_chart

print("Ambiente configurado com sucesso.")

Ambiente configurado com sucesso.


In [9]:
# ===================================================================================
# 2. CONEXÃO, PRÉ-PROCESSAMENTO E AMOSTRAGEM
# ===================================================================================
path_dados = 'dados_catalogo/processed/dados_consolidados_filtrados.parquet'
con = duckdb.connect(database=':memory:', read_only=False)
db_api = DuckDBAPI(connection=con)

# SQL para criar a VIEW com o dataset completo e as colunas de array
sql_completo = f"""
CREATE OR REPLACE VIEW dados_completos AS
SELECT *,
    list_distinct(list_sort(string_to_array(trim(BB_Cast), ','))) as BB_Cast_array,
    list_distinct(list_sort(string_to_array(trim(BB_Directors), ','))) as BB_Directors_array
FROM read_parquet('{path_dados}');
"""
con.execute(sql_completo)

# Usaremos uma amostra para todo o processo de desenvolvimento do modelo.
sql_amostra = "CREATE OR REPLACE VIEW dados_amostra AS SELECT * FROM dados_completos USING SAMPLE 30000 ROWS;"
con.execute(sql_amostra)
print("View 'dados_amostra' (30 mil linhas) criada para uma exploração rápida.")

View 'dados_amostra' (30 mil linhas) criada para uma exploração rápida.


In [11]:
# ===================================================================================
# 3. ANÁLISE EXPLORATÓRIA APROFUNDADA (NA AMOSTRA)
# ===================================================================================
# Passo 3.1: Análise de Completude (Valores Nulos)
print("--- Análise de Completude ---")
display(completeness_chart(table_or_tables="dados_amostra", db_api=db_api))

# Passo 3.2: Perfil das Colunas Chave
print("\n--- Perfil da Coluna 'BB_Title' e 'BB_Year' ---")
display(profile_columns(table_or_tables="dados_amostra", column_expressions=["BB_Title", "BB_Year"], db_api=db_api))

# Passo 3.3: Criando um Linker Preliminar para Análises Mais Complexas
linker_para_analise = Linker(
    "dados_amostra",
    SettingsCreator(link_type="dedupe_only", unique_id_column_name="BB_UID"),
    db_api=db_api
)

print("\n--- Análise Customizada: Verificando Diretores Mais Frequentes ---")
# CORREÇÃO: A sintaxe do UNNEST foi ajustada para o padrão do DuckDB,
# movendo a função para a cláusula FROM.
sql_diag_directors = """
SELECT director, COUNT(*) as frequency
FROM dados_amostra, unnest(BB_Directors_array) AS t(director)
GROUP BY director
ORDER BY frequency DESC
LIMIT 10;
"""
# Esta query agora é sintaticamente válida e executará corretamente.
display(linker_para_analise.misc.query_sql(sql_diag_directors, output_type="pandas"))

--- Análise de Completude ---


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

alt.LayerChart(...)


--- Perfil da Coluna 'BB_Title' e 'BB_Year' ---


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

alt.VConcatChart(...)


--- Análise Customizada: Verificando Diretores Mais Frequentes ---


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,director,frequency
0,Igreja Universal,294
1,Não especificado,179
2,Universal Church,135
3,Renato Ribeiro,114
4,Église Universelle,87
5,N,61
6,Iglesia Universal,58
7,Universele Kerk Van Gods Rijk,53
8,Gisele Kato,39
9,Not specified,39


In [12]:
# ===================================================================================
# 4. ANÁLISE E SELEÇÃO DE REGRAS DE BLOQUEIO (NA AMOSTRA)
# ===================================================================================
regras_de_bloqueio_candidatas = [
    block_on("BB_Title", "BB_Year"),
    block_on("substr(BB_Title, 1, 4)", "BB_Year"),
    block_on("substr(BB_Directors, 1, 6)", "BB_Year"),
    block_on("substr(BB_Cast, 1, 6)", "BB_Year"),
    block_on("BB_Title", "substr(BB_Directors, 1, 6)"),
]

print("\n--- Análise Cumulativa de Comparações para Regras Candidatas ---")
# Este gráfico é essencial para projetar a performance. Ele mostra o número de pares
# que cada regra adiciona ao processo, nos ajudando a escolher um conjunto que seja
# abrangente (alto recall) mas computacionalmente viável.
cumulative_comparisons_to_be_scored_from_blocking_rules_chart(
    table_or_tables="dados_amostra",
    blocking_rules=regras_de_bloqueio_candidatas,
    db_api=db_api,
    link_type="dedupe_only",
    unique_id_column_name="BB_UID"
)


--- Análise Cumulativa de Comparações para Regras Candidatas ---


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

alt.Chart(...)

In [17]:
# ===================================================================================
# 5. CONFIGURAÇÃO DO MODELO COM TRATAMENTO DE TIPOS DE DADOS (SINTAXE CORRIGIDA)
# ===================================================================================

settings_modelo = SettingsCreator(
    link_type="dedupe_only",
    unique_id_column_name="BB_UID",
    blocking_rules_to_generate_predictions=regras_de_bloqueio_candidatas, # Definida na Célula 4
    
    comparisons=[
        cl.JaroWinklerAtThresholds("BB_Title", [0.9, 0.7]),
        
        cl.CustomComparison(
            output_column_name="BB_Year",
            comparison_levels=[
                cll.NullLevel("BB_Year"),
                cll.ExactMatchLevel("BB_Year"),
                # Nível 2: Permite 1 erro de digitação, mas opera no texto
                # CORREÇÃO: Trocado 'column_name' por 'col_name'
                cll.DamerauLevenshteinLevel(
                    col_name="CAST(BB_Year AS VARCHAR)", 
                    distance_threshold=1
                ),
                cll.ElseLevel()
            ]
        ),
        
        cl.ArrayIntersectAtSizes("BB_Directors_array", [1]).configure(
            term_frequency_adjustments=True
        ),
        
        cl.ArrayIntersectAtSizes("BB_Cast_array", [3, 1]).configure(
            term_frequency_adjustments=True
        ),
    ],
    
    retain_matching_columns=True,
    retain_intermediate_calculation_columns=True
)

print("Configuração (settings) avançada e com tratamento de tipos de dados criada.")

Configuração (settings) avançada e com tratamento de tipos de dados criada.


In [18]:
# ===================================================================================
# 6. TREINAMENTO E ANÁLISE DOS PARÂMETROS (NA AMOSTRA)
# ===================================================================================
print("--- Criando o Linker de modelo para a amostra ---")
linker_modelo = Linker("dados_amostra", settings_modelo, db_api=db_api)

print("\n--- Estimando probabilidades 'u' ---")
linker_modelo.training.estimate_u_using_random_sampling(max_pairs=2e6)

print("\n--- Iniciando sessões de treinamento EM ---")
# Sessão 1: Foca em estimar os parâmetros para Diretores e Elenco
linker_modelo.training.estimate_parameters_using_expectation_maximisation(block_on("BB_Title", "BB_Year"))
# Sessão 2: Foca em estimar os parâmetros para Título
linker_modelo.training.estimate_parameters_using_expectation_maximisation(block_on("BB_Directors", "BB_Year"))
print("\nTreinamento do modelo concluído!")

print("\n--- Comparando as estimativas dos parâmetros das diferentes sessões de EM ---")
# Este gráfico é uma ótima ferramenta de diagnóstico. Se as estimativas para a mesma
# comparação (ex: BB_Cast_array) são muito diferentes entre as sessões, pode
# indicar que as regras de bloqueio do treinamento estão enviesadas.
display(linker_modelo.visualisations.parameter_estimate_comparisons_chart())

----- Estimating u probabilities using random sampling -----


--- Criando o Linker de modelo para a amostra ---

--- Estimando probabilidades 'u' ---



Estimated u probabilities using random sampling

Your model is not yet fully trained. Missing estimates for:
    - BB_Title (no m values are trained).
    - BB_Year (no m values are trained).
    - BB_Directors_array (no m values are trained).
    - BB_Cast_array (no m values are trained).



--- Iniciando sessões de treinamento EM ---


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))


----- Starting EM training session -----

Estimating the m probabilities of the model by blocking on:
(l."BB_Title" = r."BB_Title") AND (l."BB_Year" = r."BB_Year")

Parameter estimates will be made for the following comparison(s):
    - BB_Directors_array
    - BB_Cast_array

Parameter estimates cannot be made for the following comparison(s) since they are used in the blocking rules: 
    - BB_Title
    - BB_Year

Iteration 1: Largest change in params was 0.175 in probability_two_random_records_match
Iteration 2: Largest change in params was 0.178 in probability_two_random_records_match
Iteration 3: Largest change in params was 0.0256 in probability_two_random_records_match
Iteration 4: Largest change in params was 0.00367 in probability_two_random_records_match
Iteration 5: Largest change in params was 0.000526 in probability_two_random_records_match
Iteration 6: Largest change in params was 7.54e-05 in probability_two_random_records_match

EM converged after 6 iterations

Your model


Treinamento do modelo concluído!

--- Comparando as estimativas dos parâmetros das diferentes sessões de EM ---


alt.Chart(...)

In [21]:
# ===================================================================================
# 7. VITRINE DE FERRAMENTAS DE VISUALIZAÇÃO E AVALIAÇÃO (NA AMOSTRA)
# ===================================================================================

# --- 7.1: Análise Visual dos Pesos do Modelo ---
print("--- Gráfico de Pesos de Correspondência (Match Weights) ---")
display(linker_modelo.visualisations.match_weights_chart())

# --- 7.2: Predição e Análise de Casos Específicos ---
print("\n--- Executando predição na amostra ---")
df_predict_amostra = linker_modelo.inference.predict()

print("\n--- Gráfico Waterfall: Dissecando as Predições ---")
records_para_analise = df_predict_amostra.as_record_dict(limit=3)

nome_tabela_predict = df_predict_amostra.physical_name
borderline_sql = f"SELECT * FROM {nome_tabela_predict} ORDER BY abs(match_weight) LIMIT 3"

# CORREÇÃO: Primeiro, obtemos o resultado como um DataFrame do pandas.
borderline_df = linker_modelo.misc.query_sql(borderline_sql, output_type="pandas")
# Em seguida, convertemos o DataFrame do pandas para o formato de lista de dicionários.
borderline_records = borderline_df.to_dict(orient="records")
records_para_analise.extend(borderline_records)

# Agora, o waterfall chart recebe a lista de dicionários no formato correto.
display(linker_modelo.visualisations.waterfall_chart(records_para_analise))

# --- 7.3: Dashboards Interativos ---
print("\n--- Gerando o Dashboard 'Comparison Viewer' ---")
linker_modelo.visualisations.comparison_viewer_dashboard(
    df_predict_amostra, "comparison_viewer.html", overwrite=True
)
print("Dashboard salvo em 'comparison_viewer.html'. Abra-o para uma análise interativa.")

# --- 7.4: Clusterização e Análise de Clusters ---
print("\n--- Clusterizando os resultados da amostra ---")
clusters_amostra = linker_modelo.clustering.cluster_pairwise_predictions_at_threshold(df_predict_amostra, 0.9)

print("\n--- Gerando o Dashboard 'Cluster Studio' ---")
linker_modelo.visualisations.cluster_studio_dashboard(
    df_predict_amostra, clusters_amostra, "cluster_studio.html", overwrite=True
)
print("Dashboard salvo em 'cluster_studio.html'.")

--- Gráfico de Pesos de Correspondência (Match Weights) ---


alt.VConcatChart(...)


--- Executando predição na amostra ---


Blocking time: 0.17 seconds
Predict time: 0.06 seconds

 -- WARNING --
You have called predict(), but there are some parameter estimates which have neither been estimated or specified in your settings dictionary.  To produce predictions the following untrained trained parameters will use default values.
Comparison: 'BB_Year':
    m values not fully trained
The 'probability_two_random_records_match' setting has been set to the default value (0.0001). 
If this is not the desired behaviour, either: 
 - assign a value for `probability_two_random_records_match` in your settings dictionary, or 
 - estimate with the `linker.estimate_probability_two_random_records_match` function.



--- Gráfico Waterfall: Dissecando as Predições ---


alt.LayerChart(...)


--- Gerando o Dashboard 'Comparison Viewer' ---


Completed iteration 1, num representatives needing updating: 17
Completed iteration 2, num representatives needing updating: 2
Completed iteration 3, num representatives needing updating: 0


Dashboard salvo em 'comparison_viewer.html'. Abra-o para uma análise interativa.

--- Clusterizando os resultados da amostra ---

--- Gerando o Dashboard 'Cluster Studio' ---
Dashboard salvo em 'cluster_studio.html'.


In [22]:
# ===================================================================================
# 8. CONCLUSÃO E CAMINHO PARA PRODUÇÃO
# ===================================================================================

# O objeto 'linker_modelo' contém as configurações e os parâmetros treinados
# a partir da amostra. A primeira ação é salvar este modelo (settings + parâmetros)
# em um arquivo JSON. Este arquivo é o principal artefato do nosso trabalho.
caminho_modelo = "modelo_vod_exploratorio_v1.json"
linker_modelo.misc.save_model_to_json(caminho_modelo, overwrite=True)
print(f"Modelo treinado na amostra foi salvo com sucesso em: {caminho_modelo}")

print("\n===================================================================================")
print("FLUXO DE TRABALHO EXPLORATÓRIO CONCLUÍDO")
print("O modelo foi projetado, treinado e validado na amostra de dados.")
print("===================================================================================")


# O CÓDIGO ABAIXO É UM GUIA PARA A PRÓXIMA FASE: APLICAR O MODELO NO DATASET COMPLETO.
# ELE ESTÁ COMENTADO E NÃO DEVE SER EXECUTADO NESTE NOTEBOOK, MAS SIM EM UM NOVO SCRIPT/NOTEBOOK DE PRODUÇÃO.

#
# print("\n--- GUIA PARA APLICAR EM PRODUÇÃO (NÃO EXECUTAR AQUI) ---")
#
# # 1. Em um novo ambiente, crie o Linker apontando para os dados completos
# #    e carregue o modelo que acabamos de salvar.
# from splink import Linker, DuckDBAPI
#
# db_api_prod = DuckDBAPI(connection=con) # Reutilize a conexão se estiver no mesmo ambiente
# linker_producao = Linker("dados_completos", caminho_modelo, db_api=db_api_prod)
# print("Linker de produção criado, carregando modelo a partir do arquivo.")
#
# # 2. (OPCIONAL, MAS RECOMENDADO) Re-treinar os parâmetros 'm' no dataset completo.
# #    Os parâmetros 'u' e a estrutura do modelo já estão bons, mas re-treinar 'm'
# #    ajusta o modelo à distribuição completa dos dados. Isso pode ser rápido se
# #    as regras de bloqueio forem eficientes.
#
# print("\nRe-treinando rapidamente os parâmetros 'm' no dataset completo...")
# linker_producao.training.estimate_parameters_using_expectation_maximisation(block_on("BB_Title", "BB_Year"))
# linker_producao.training.estimate_parameters_using_expectation_maximisation(block_on("BB_Directors", "BB_Year"))
#
# # 3. Executar a predição e clusterização final no dataset completo.
# #    Esta será a etapa mais demorada do processo de produção.
#
# print("\nExecutando a predição final no dataset completo...")
# df_predict_final = linker_producao.inference.predict()
#
# print("\nGerando a Tabela de Vínculos final...")
# clusters_finais = linker_producao.clustering.cluster_pairwise_predictions_at_threshold(df_predict_final, 0.9)
#
# # 4. Salvar os resultados finais para uso no seu pipeline.
#
# print("\nSalvando a Tabela de Vínculos em um arquivo Parquet...")
# df_final_pandas = clusters_finais.as_pandas_dataframe()
# df_final_pandas.to_parquet("tabela_de_vinculos_final.parquet")
# print("Processo de produção concluído. A tabela 'tabela_de_vinculos_final.parquet' está pronta.")

Modelo treinado na amostra foi salvo com sucesso em: modelo_vod_exploratorio_v1.json

FLUXO DE TRABALHO EXPLORATÓRIO CONCLUÍDO
O modelo foi projetado, treinado e validado na amostra de dados.
